In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd

In [2]:
regions = gpd.read_parquet('/data/uscuni-ulce/regions/cadastre_regions_hull.parquet')


In [8]:
!unzip PMO\ ITI.zip

Archive:  PMO ITI.zip
 extracting: PMO ITI/praзskа metropolitnб oblast.cpg  
  inflating: PMO ITI/praзskа metropolitnб oblast.dbf  
  inflating: PMO ITI/praзskа metropolitnб oblast.prj  
  inflating: PMO ITI/praзskа metropolitnб oblast.sbn  
  inflating: PMO ITI/praзskа metropolitnб oblast.sbx  
  inflating: PMO ITI/praзskа metropolitnб oblast.shp  
  inflating: PMO ITI/praзskа metropolitnб oblast.shp.xml  
  inflating: PMO ITI/praзskа metropolitnб oblast.shx  


In [62]:
bounds = gpd.read_file('PMO ITI/').to_crs(epsg=3035)
bounds = bounds.union_all().exterior
bounds = Polygon(bounds.coords)

In [86]:
%%time

bounds_etcs = []

for rid in bounds_regions:

    region_buildings = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/buildings_chars_{rid}.parquet')
    region_buildings_within_bounds = region_buildings.index[region_buildings.sindex.query(bounds, predicate='intersects')].values
    
    region_chars = pd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/primary_chars_{rid}.parquet')
    region_chars = region_chars.loc[region_buildings_within_bounds]

    to_save = gpd.GeoDataFrame(region_chars, 
                 geometry=region_buildings.loc[region_buildings_within_bounds].geometry,
                 crs=region_buildings.crs)

    to_save.index = str(rid) + '_' + to_save.index.astype(str)
    
    bounds_etcs.append(to_save)

CPU times: user 3.08 s, sys: 945 ms, total: 4.03 s
Wall time: 1.83 s


In [88]:
all_buildings = pd.concat(bounds_etcs)

In [93]:
all_buildings.to_parquet('/data/uscuni-ulce/processed_data/outputs/prague/pmo_chars.pq')